In [1]:
from functions import *
from torch.optim import Adam
from functions_pt import *

In [2]:
embed_size = 64
dropout = 0.1
multiple_factor = 1
layers = 4
batch_size = 64
concat = True
agg = 'mean' # mean,sum,min,max
model = pointNetQ(layers,embed_size,agg,dropout,multiple_factor,concat)#.to('cuda')
#model.load_state_dict(torch.load('baseline_noBN.pt'))
#model.eval()
model = model.to('cuda')

In [3]:
words_embed = np.load('possible_words_embed.npy')
allowed_words_embed = np.load('allowed_words_embed.npy')

RL Loop

In [ ]:
# model trained from start
lr = 6e-5
clip = 0.03
epochs = 3
damp = 1
iterations = 30
max_data = 2000
states = set()
opt = Adam(model.parameters(),betas=(0.85, 0.95),lr=lr)
best_score = 100
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_saveQ(matrix,np.arange(2309),policy_modelQ,2309,0,model=model,words_embed=words_embed,
                           allowed_words_embed=allowed_words_embed,top=0.6,eps=0.08/(i+1))
    print('iter:{}, score:{}'.format(i,score))
    if score < best_score:
        best_score = score
        best_weight = copy.deepcopy(model.state_dict()) 
    # prepare data for training
    index,val,log_prob,action,entro = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],device='cuda').to(torch.long)
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    entro = torch.tensor(entro,dtype=torch.float32,device='cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')
    actions_tch = torch.tensor(allowed_words_embed[np.array(action)],device='cuda').long()
    
    # policy update
    model.train()
    loss_int,counter_ = 1000,0
    tmpWeight = copy.deepcopy(model.state_dict())   
    while counter_ < epochs:
        yhat = model((words,length,actions_tch,entro))
        loss = torch.sum(prob*(ys-yhat)**2)/torch.sum(prob)
        print('loss:{}'.format(loss.item()))
        if loss.item() > loss_int + 0.1:
            model.load_state_dict(tmpWeight)
            for g in opt.param_groups:
                g['lr'] = g['lr']/1.5
        else:
            loss_int = loss.item()
            counter_ += 1
            tmpWeight = copy.deepcopy(model.state_dict())
            loss.backward()
            clip_grad_value_(model.parameters(),clip)
            opt.step()
            opt.zero_grad()
    for g in opt.param_groups:
        g['lr'] = lr
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)
model.load_state_dict(best_weight)

iter:0, score:3.7717626678215703
set size:421
loss:2.2254791259765625
loss:2.1844828128814697
loss:1.9852381944656372
iter:1, score:3.792117799913384
set size:835
loss:1.8369802236557007
loss:1.4951525926589966
loss:1.2510122060775757
iter:2, score:3.7553053269813788
set size:1232
loss:1.2899497747421265
loss:0.8646899461746216
loss:0.6360717415809631
iter:3, score:3.779558250324816
set size:1640
loss:0.5028873682022095
loss:0.43472540378570557
loss:0.3668586015701294
iter:4, score:3.777825898657428
set size:2042
loss:0.3637322783470154
loss:0.6587222218513489
loss:0.36804118752479553
loss:0.3786114454269409
iter:5, score:3.7566045907319205
set size:2407
loss:0.37180978059768677
loss:0.4154185354709625
loss:0.6259401440620422
loss:0.37562495470046997
iter:6, score:3.8142052836725884
set size:2802
loss:0.5898191928863525
loss:0.5825686454772949
loss:0.5436686277389526


In [5]:
torch.save(model.state_dict(), 'RL_Q0.pt')

In [6]:
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6,
                         eps=0)

3.437418796015592